<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# `GiRaFFE_NRPy_staggered`: Source Terms

## Author: Patrick Nelson

**Notebook Status:** <font color='green'><b>Validated</b></font>

**Validation Notes:** This code is a port from the old `GiRaFFE`

### NRPy+ Source Code for this module:
* [GiRaFFE_NRPy/GiRaFFE_NRPy_staggered_Source_Terms.py](../../edit/in_progress/GiRaFFE_NRPy/GiRaFFE_NRPy_staggered_Source_Terms.py)

## Introduction: 

This notebook presents an alternate algorithm for computing the source terms in a staggered prescription. It is a direct port of the old `GiRaFFE` implementation.

Our goal here will to be to add the following terms to the appropriate right hand side: 
* $\partial_t A_i -= \partial_i (\alpha \Phi - \beta^j A_j)$
* $\partial_t [\sqrt{\gamma} \Phi] + \partial_j (\alpha\sqrt{\gamma}A^j - \beta^j [\sqrt{\gamma} \Phi]) = - \xi \alpha [\sqrt{\gamma} \Phi]$
(Note, the *whole* of the $\partial_t [\sqrt{\gamma} \Phi]$ evolution equation is of interest here.)

This will be done using basic second-order finite-difference derivatives; however, there is a significant wrinkle here: Not all of these quantities are sampled at cell centers! In particular, at code index `i,j,k`, the following quantities are what is actually available:
* Centers: $B^x(i,j,k)$, $B^y(i,j,k)$, $B^z(i,j,k)$
* Faces: $B^x(i+1/2,j,k)$, $B^y(i,j+1/2,k)$, $B^z(i,j,k+1/2)$
* Edges: $A_x(i,j+1/2,k+1/2)$, $A_y(i+1/2,j,k+1/2)$, $A_z(i+1/2,j+1/2,k)$
* Vertices: $[\sqrt{\gamma} \Phi](i+1/2,j+1/2,k+1/2)$

(Unless otherwise specified above, the quantity is available at cell centers.)

To overcome this, we will make heavy use of interpolation; it is sufficient for our purposes to merely average over the nearest points. 

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This notebook is organized as follows

1. [Step 1](#prelim): Preliminaries
1. [Step 2](#code): Write the C code
    1. [Step 2.a](#define_constants): Function definitions and useful constants
    1. [Step 2.b](#operands): Compute derivative operands
        1. [Step 2.b.i](#interp): Read in quantities needed for interpolation

        1. [Step 2.b.ii](#raise_a): Compute $\alpha \sqrt{\gamma} A^j$

        1. [Step 2.b.iii](#a_gauge_term): Compute $\alpha \Phi - \beta^j A_j$

    1. [Step 2.c](#finite_difference): Take the finite difference derivatives

        1. [Step 2.c.i](#a_rhs): Compute the $A_i$ gauge term
        1. [Step 2.c.ii](#phi_rhs): Compute the $[\psi^6 \Phi]$ gauge term

    1. [Step 2.d](#interp_func): The interpolation function
1. [Step 3](#code_validation): Code Validation
1. [Step 4](#latex_pdf_output): Output this notebook to $\LaTeX$-formatted PDF file

<a id='prelim'></a>

# Step 1: Preliminaries \[Back to [top](#toc)\]
$$\label{prelim}$$

This first block of code just sets up a subdirectory within `GiRaFFE_standalone_Ccodes/` to which we will write the C code.

In [1]:
# Step 0: Add NRPy's directory to the path
# https://stackoverflow.com/questions/16780014/import-file-from-parent-directory
import os,sys
nrpy_dir_path = os.path.join("..")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)

import cmdline_helper as cmd     # NRPy+: Multi-platform Python command-line interface
Ccodesdir = "GiRaFFE_standalone_Ccodes/RHSs"
cmd.mkdir(os.path.join(Ccodesdir))

<a id='code'></a>

# Step 2: Write the C code \[Back to [top](#toc)\]
$$\label{code}$$

<a id='define_constants'></a>

## Step 2.a: Function definitions and useful constants \[Back to [top](#toc)\]
$$\label{define_constants}$$

We will first declare the crucial function `avg()`, our interpolator; its inputs are a grid function as will as the limits over which we will loop. [It is defined below](#interp_func).

We will also `#define` some constants. We will frequently need to loop *near* a given gridpoint by up to two gridpoints in any direction, so these constants will allow us to define these limits more intuitively. The rest of these provide similar heuristics to access memory within the construct `in_vars` and `INTERP_VARS` that we will use to pass data without a large mess of function inputs.

In [2]:
%%writefile $Ccodesdir/Lorenz_psi6phi_rhs__add_gauge_terms_to_A_i_rhs.h
static inline REAL avg(const REAL f[PLUS2+1][PLUS2+1][PLUS2+1],const int imin,const int imax, const int jmin,const int jmax, const int kmin,const int kmax);

#define MINUS2 0
#define MINUS1 1
#define PLUS0  2
#define PLUS1  3
#define PLUS2  4
// The "I" suffix denotes interpolation. In other words, these
//    definitions are used for interpolation ONLY. The order here
//    matters as well!
static const int SHIFTXI=0,SHIFTYI=1,SHIFTZI=2,GUPXXI=3,GUPXYI=4,GUPXZI=5,GUPYYI=6,GUPYZI=7,GUPZZI=8,
  PSII=9,LAPM1I=10,A_XI=11,A_YI=12,A_ZI=13,LAPSE_PSI2I=14,LAPSE_OVER_PSI6I=15;
#define MAXNUMINTERP 16
// GiRaFFE_NRPy does not store the inverse metric. So, the actual inputs to the function will be
// just the metric, which we can invert in an early step. To keep things consistent, we'll label the
// components with the same labels as the inverse:
static const int GXXI=3,GXYI=4,GXZI=5,GYYI=6,GYZI=7,GZZI=8;


Our function includes the C parameters `params`, an array of pointers to our input variables `in_vars`, the input `psi6phi`, eight arrays for temporary variables, and the output arrays for the potential.

Finally, we also declare variables from the paramstruct in the normal way with the `#include` directive. (If you're reading this code and can't figure out where a variable is set, check here!)

In [3]:
%%writefile -a $Ccodesdir/Lorenz_psi6phi_rhs__add_gauge_terms_to_A_i_rhs.h

static void Lorenz_psi6phi_rhs__add_gauge_terms_to_A_i_rhs(const paramstruct *params,REAL **in_vars,const REAL *psi6phi,
                                                           /* TEMPS: */
                                                           REAL *shiftx_iphjphkph,REAL *shifty_iphjphkph,REAL *shiftz_iphjphkph,
                                                           REAL *alpha_iphjphkph,REAL *alpha_Phi_minus_betaj_A_j_iphjphkph,REAL *alpha_sqrtg_Ax_interp,
                                                           REAL *alpha_sqrtg_Ay_interp,REAL *alpha_sqrtg_Az_interp,
                                                           /* END TEMPS, 8 total! */
                                                           REAL *psi6phi_rhs,REAL *Ax_rhs,REAL *Ay_rhs,REAL *Az_rhs) {
  #include "../set_Cparameters.h"


<a id='operands'></a>

## Step 2.b: Compute derivative operands \[Back to [top](#toc)\]
$$\label{operands}$$

We will first compute $ \partial_t [\psi^6\Phi] = -\partial_j ( \alpha \sqrt{\gamma} A^j - \beta^j \psi^6\Phi)$, taken from Eq. 13 of [this](https://arxiv.org/pdf/1110.4633.pdf) paper (the Lorentz gauge evolution). Since all input variables to this are defined at all gridpoints, we can safely start out by looping over the entire grid *except* for the outermost ghostzone in each direction. We will define the index in our data storage of the point $(i,j,k)$ for the iteration and declare storage for the values that we will use to do the interpolation.

In [4]:
%%writefile -a $Ccodesdir/Lorenz_psi6phi_rhs__add_gauge_terms_to_A_i_rhs.h

  /* Compute
   * \partial_t psi6phi = -\partial_j ( \alpha \sqrt{\gamma} A^j - \beta^j psi6phi)
   *    (Eq 13 of http://arxiv.org/pdf/1110.4633.pdf), using Lorenz gauge.
   * Note that the RHS consists of a shift advection term on psi6phi and
   *    a term depending on the vector potential.
   * psi6phi is defined at (i+1/2,j+1/2,k+1/2), but instead of reconstructing
   *    to compute the RHS of \partial_t psi6phi, we instead use standard
   *    interpolations.
   */

  // The stencil here is {-1,1},{-1,1},{-1,1} for x,y,z directions, respectively.
  //     Note that ALL input variables are defined at ALL gridpoints, so no
  //     worries about ghostzones.
#pragma omp parallel for
  for(int k=1;k<Nxx_plus_2NGHOSTS2-1;k++) for(int j=1;j<Nxx_plus_2NGHOSTS1-1;j++) for(int i=1;i<Nxx_plus_2NGHOSTS0-1;i++) {
        const int index=IDX3S(i,j,k);
        REAL INTERP_VARS[MAXNUMINTERP][PLUS2+1][PLUS2+1][PLUS2+1];


Appending to GiRaFFE_standalone_Ccodes/RHSs/Lorenz_psi6phi_rhs__add_gauge_terms_to_A_i_rhs.h


<a id='interp'></a>

### Step 2.b.i: Read in quantities needed for interpolation  \[Back to [top](#toc)\]
$$\label{interp}$$

We will code this one term at a time, starting with $\partial_j (\alpha \sqrt{\gamma} A^j)$. To do so, we will need to find this quantity at the cell vertex $(i+1/2,j+1/2,k+1/2)$. Since $A_x$ exists at $(i,j+1/2,k+1/2)$ it makes sense to do this by finding $(\alpha \sqrt{\gamma} A^x)$ at $(i,j+1/2,k+1/2)$ and $(i+1,j+1/2,k+1/2)$ and using those values to perform the finite differencing. (And so on, for the other components of $A_i$.)

Note also that $\alpha \sqrt{\gamma} A^j = \alpha \sqrt{\gamma} \gamma^{ij} A_i$; so, we will need to interpolate the inverse metric $\gamma^{ij}$, the square root of the metric determinant $\sqrt{\gamma}$, and the lapse function $\alpha$ to the cell edges. 

So, we must prepare for all these interpolations. We will read in the values of the above specified metric gridfunctions in a small cube two gridpoints in every direction and write them to `INTERP_VARS`.

In [5]:
%%writefile -a $Ccodesdir/Lorenz_psi6phi_rhs__add_gauge_terms_to_A_i_rhs.h

        // First compute \partial_j \alpha \sqrt{\gamma} A^j (RHS of \partial_i psi6phi)
        // FIXME: Would be much cheaper & easier to unstagger A_i, raise, then interpolate A^i.
        //        However, we keep it this way to be completely compatible with the original
        //        Illinois GRMHD thorn, called mhd_evolve.
        //
        //Step 1) j=x: Need to raise A_i, but to do that, we must have all variables at the same gridpoints:
        // The goal is to compute \partial_j (\alpha \sqrt{\gamma} A^j) at (i+1/2,j+1/2,k+1/2)
        //    We do this by first interpolating (RHS1x) = (\alpha \sqrt{\gamma} A^x) at
        //    (i,j+1/2,k+1/2)and (i+1,j+1/2,k+1/2), then taking \partial_x (RHS1x) =
        //    [ RHS1x(i+1,j+1/2,k+1/2) - RHS1x(i,j+1/2,k+1/2) ]/dX.
        // First bring gup's, psi, and alpha to (i,j+1/2,k+1/2):
        int num_vars_to_interp;
        int vars_to_interpolate[MAXNUMINTERP] = {GUPXXI,GUPXYI,GUPXZI,GUPYYI,GUPYZI,GUPZZI,LAPM1I,PSII,SHIFTXI,SHIFTYI,SHIFTZI};
        num_vars_to_interp = 11;
        // We may set interp_limits to be more general than we need.
        int interp_limits[6] = {-1,1,-1,1,-1,1}; SET_INDEX_ARRAYS_NRPY_3DBLOCK(interp_limits);
        // Major change here to invert the metric on the spot!
        // Read in variable at interp. stencil points from main memory, store in INTERP_VARS.
        for(int kk=PLUS0;kk<=PLUS1;kk++) for(int jj=PLUS0;jj<=PLUS1;jj++) for(int ii=PLUS0;ii<=PLUS1;ii++) {
            // First, we will read in each component of the metric, then find the determinant.
            // We write the twelfth root to psi_bssn. Then, we invert the metric and store these values.
            const REAL gammaDD00 = in_vars[GXXI][index_arr_3DB[kk][jj][ii]];
            const REAL gammaDD01 = in_vars[GXYI][index_arr_3DB[kk][jj][ii]];
            const REAL gammaDD02 = in_vars[GXZI][index_arr_3DB[kk][jj][ii]];
            const REAL gammaDD11 = in_vars[GYYI][index_arr_3DB[kk][jj][ii]];
            const REAL gammaDD12 = in_vars[GYZI][index_arr_3DB[kk][jj][ii]];
            const REAL gammaDD22 = in_vars[GZZI][index_arr_3DB[kk][jj][ii]];
            // Generated by NRPy+:
            /*
             * NRPy+ Finite Difference Code Generation, Step 2 of 1: Evaluate SymPy expressions and write to main memory:
             */
            const double FDPart3_0 = cbrt(gammaDD00*gammaDD11*gammaDD22 - gammaDD00*((gammaDD12)*(gammaDD12)) - ((gammaDD01)*(gammaDD01))*gammaDD22 + 2*gammaDD01*gammaDD02*gammaDD12 - ((gammaDD02)*(gammaDD02))*gammaDD11);
            const double FDPart3_1 = (1.0/(FDPart3_0));
            const REAL gamma_bssnDD00 = FDPart3_1*gammaDD00;
            const REAL gamma_bssnDD01 = FDPart3_1*gammaDD01;
            const REAL gamma_bssnDD02 = FDPart3_1*gammaDD02;
            const REAL gamma_bssnDD11 = FDPart3_1*gammaDD11;
            const REAL gamma_bssnDD12 = FDPart3_1*gammaDD12;
            const REAL gamma_bssnDD22 = FDPart3_1*gammaDD22;
            const double tmp_5 = gamma_bssnDD00*gamma_bssnDD11*gamma_bssnDD22 - gamma_bssnDD00*((gamma_bssnDD12)*(gamma_bssnDD12)) - ((gamma_bssnDD01)*(gamma_bssnDD01))*gamma_bssnDD22 + 2*gamma_bssnDD01*gamma_bssnDD02*gamma_bssnDD12 - ((gamma_bssnDD02)*(gamma_bssnDD02))*gamma_bssnDD11;
            const double tmp_6 = (1.0/(tmp_5));
            INTERP_VARS[GUPXXI][kk][jj][ii] = tmp_6*(gamma_bssnDD11*gamma_bssnDD22 - ((gamma_bssnDD12)*(gamma_bssnDD12)));
            INTERP_VARS[GUPXYI][kk][jj][ii] = tmp_6*(-gamma_bssnDD01*gamma_bssnDD22 + gamma_bssnDD02*gamma_bssnDD12);
            INTERP_VARS[GUPXZI][kk][jj][ii] = tmp_6*(gamma_bssnDD01*gamma_bssnDD12 - gamma_bssnDD02*gamma_bssnDD11);
            INTERP_VARS[GUPYYI][kk][jj][ii] = tmp_6*(gamma_bssnDD00*gamma_bssnDD22 - ((gamma_bssnDD02)*(gamma_bssnDD02)));
            INTERP_VARS[GUPYZI][kk][jj][ii] = tmp_6*(-gamma_bssnDD00*gamma_bssnDD12 + gamma_bssnDD01*gamma_bssnDD02);
            INTERP_VARS[GUPZZI][kk][jj][ii] = tmp_6*(gamma_bssnDD00*gamma_bssnDD11 - ((gamma_bssnDD01)*(gamma_bssnDD01)));
            INTERP_VARS[PSII][kk][jj][ii]   = pow(FDPart3_0,1.0/4.0);

            // Now, we read in the lapse function.
            int whichvar=vars_to_interpolate[6];
            INTERP_VARS[whichvar][kk][jj][ii] = in_vars[whichvar][index_arr_3DB[kk][jj][ii]]-1.0; // Input alpha, expect alpha-1
            // Finally, we read in the shift vector into the array.
            for(int ww=8;ww<num_vars_to_interp;ww++) {
                int whichvar=vars_to_interpolate[ww];
                INTERP_VARS[whichvar][kk][jj][ii] = in_vars[whichvar][index_arr_3DB[kk][jj][ii]];
            }
        }


Appending to GiRaFFE_standalone_Ccodes/RHSs/Lorenz_psi6phi_rhs__add_gauge_terms_to_A_i_rhs.h


While we are at it, we will also interpolate the lapse function $\alpha$ to the cell vertices. 

In [6]:
%%writefile -a $Ccodesdir/Lorenz_psi6phi_rhs__add_gauge_terms_to_A_i_rhs.h

        // Next set \alpha at (i+1/2,j+1/2,k+1/2). Will come in handy when computing damping term later.
        alpha_iphjphkph[index] = avg(INTERP_VARS[LAPM1I] , PLUS0,PLUS1, PLUS0,PLUS1, PLUS0,PLUS1)+1.0;


Appending to GiRaFFE_standalone_Ccodes/RHSs/Lorenz_psi6phi_rhs__add_gauge_terms_to_A_i_rhs.h


Here, we set up the interpolation stencils for the vector potential. These are defined by the maximum extents of the interpolations that we do below, where the bounds are explained more depth.

In [7]:
%%writefile -a $Ccodesdir/Lorenz_psi6phi_rhs__add_gauge_terms_to_A_i_rhs.h

        //A_x needs a stencil s.t. interp_limits={0,1,-1,1,-1,1}:
        for(int kk=MINUS1;kk<=PLUS1;kk++) for(int jj=MINUS1;jj<=PLUS1;jj++) for(int ii=PLUS0;ii<=PLUS1;ii++) {
              INTERP_VARS[A_XI][kk][jj][ii] = in_vars[A_XI][index_arr_3DB[kk][jj][ii]]; }
        //A_y needs a stencil s.t. interp_limits={-1,1,0,1,-1,1}:
        for(int kk=MINUS1;kk<=PLUS1;kk++) for(int jj=PLUS0;jj<=PLUS1;jj++) for(int ii=MINUS1;ii<=PLUS1;ii++) {
              INTERP_VARS[A_YI][kk][jj][ii] = in_vars[A_YI][index_arr_3DB[kk][jj][ii]]; }
        //A_z needs a stencil s.t. interp_limits={-1,1,-1,1,0,1}:
        for(int kk=PLUS0;kk<=PLUS1;kk++) for(int jj=MINUS1;jj<=PLUS1;jj++) for(int ii=MINUS1;ii<=PLUS1;ii++) {
              INTERP_VARS[A_ZI][kk][jj][ii] = in_vars[A_ZI][index_arr_3DB[kk][jj][ii]]; }


Appending to GiRaFFE_standalone_Ccodes/RHSs/Lorenz_psi6phi_rhs__add_gauge_terms_to_A_i_rhs.h


<a id='raise_a'></a>

### Step 2.b.ii: Compute $\alpha \sqrt{\gamma} A^j$ \[Back to [top](#toc)\]
$$\label{raise_a}$$

Since we are starting with $A_x$, we care about the edge $(i,j+1/2,k+1/2)$ right now. As was stated above, our interpolation is an average over the nearest points, which will be $(i,j,k)$, $(i,j+1,k)$, $(i,j,k+1)$, and $(i,j+1,k+1)$ for the metric gridfunctions, which are already at cell centers.

We will also need $A_i$ at $(i,j+1/2,k+1/2)$. Because $A_x$ is already there, nothing needs to be done with it. We have $A_y(i+1/2,j,k+1/2)$, so we must average `i-1,j,k`, `i,j,k`, `i-1,j+1,k`, and `i,j+1,k`. We have $A_z(i+1/2,j+1/2,k)$, so we must average `i-1,j,k`, `i,j,k`, `i-1,j,k+1`, and `i,j,k+1`. 

Then, we are free to calculate $\alpha \sqrt{\gamma} A^x = \alpha \sqrt{\gamma} (\gamma^{xx} A_x + \gamma^{xy} A_y + \gamma^{xz} A_z)$ at every gridpoint.

In [8]:
%%writefile -a $Ccodesdir/Lorenz_psi6phi_rhs__add_gauge_terms_to_A_i_rhs.h

        // FIRST DO A^X TERM (interpolate to (i,j+1/2,k+1/2) )
        // \alpha \sqrt{\gamma} A^x = \alpha psi^6 A^x (RHS of \partial_i psi6phi)
        // Note that gupij is \tilde{\gamma}^{ij}, so we need to multiply by \psi^{-4}.
        const REAL gupxx_jphkph = avg(INTERP_VARS[GUPXXI], PLUS0,PLUS0, PLUS0,PLUS1, PLUS0,PLUS1);
        const REAL gupxy_jphkph = avg(INTERP_VARS[GUPXYI], PLUS0,PLUS0, PLUS0,PLUS1, PLUS0,PLUS1);
        const REAL gupxz_jphkph = avg(INTERP_VARS[GUPXZI], PLUS0,PLUS0, PLUS0,PLUS1, PLUS0,PLUS1);

        for(int kk=PLUS0;kk<=PLUS1;kk++) for(int jj=PLUS0;jj<=PLUS1;jj++) for(int ii=PLUS0;ii<=PLUS1;ii++) {
              const REAL Psi2 = INTERP_VARS[PSII][kk][jj][ii]*INTERP_VARS[PSII][kk][jj][ii];
              const REAL alpha = INTERP_VARS[LAPM1I][kk][jj][ii]+1.0;
              INTERP_VARS[LAPSE_PSI2I][kk][jj][ii]=alpha*Psi2;
              INTERP_VARS[LAPSE_OVER_PSI6I][kk][jj][ii]=alpha/(Psi2*Psi2*Psi2);
            }

        const REAL lapse_Psi2_jphkph = avg(INTERP_VARS[LAPSE_PSI2I], PLUS0,PLUS0, PLUS0,PLUS1, PLUS0,PLUS1);

        const REAL A_x_jphkph   = avg(INTERP_VARS[A_XI], PLUS0,PLUS0, PLUS0,PLUS0, PLUS0,PLUS0); // @ (i,j+1/2,k+1/2)
        const REAL A_y_jphkph   = avg(INTERP_VARS[A_YI],MINUS1,PLUS0, PLUS0,PLUS1, PLUS0,PLUS0); // @ (i+1/2,j,k+1/2)
        const REAL A_z_jphkph   = avg(INTERP_VARS[A_ZI],MINUS1,PLUS0, PLUS0,PLUS0, PLUS0,PLUS1); // @ (i+1/2,j+1/2,k)

        alpha_sqrtg_Ax_interp[index] = lapse_Psi2_jphkph*
          ( gupxx_jphkph*A_x_jphkph + gupxy_jphkph*A_y_jphkph + gupxz_jphkph*A_z_jphkph );



Appending to GiRaFFE_standalone_Ccodes/RHSs/Lorenz_psi6phi_rhs__add_gauge_terms_to_A_i_rhs.h


As above, except we are now calculating $\alpha \sqrt{\gamma} A^y = \alpha \sqrt{\gamma} (\gamma^{yx} A_x + \gamma^{yy} A_y + \gamma^{yz} A_z)$ at every gridpoint, making the appropriate cyclic permutations.

In [9]:
%%writefile -a $Ccodesdir/Lorenz_psi6phi_rhs__add_gauge_terms_to_A_i_rhs.h

        // DO A^Y TERM (interpolate to (i+1/2,j,k+1/2) )
        // \alpha \sqrt{\gamma} A^y = \alpha psi^6 A^y (RHS of \partial_i psi6phi)
        // Note that gupij is \tilde{\gamma}^{ij}, so we need to multiply by \psi^{-4}.
        const REAL gupxy_iphkph = avg(INTERP_VARS[GUPXYI], PLUS0,PLUS1, PLUS0,PLUS0, PLUS0,PLUS1);
        const REAL gupyy_iphkph = avg(INTERP_VARS[GUPYYI], PLUS0,PLUS1, PLUS0,PLUS0, PLUS0,PLUS1);
        const REAL gupyz_iphkph = avg(INTERP_VARS[GUPYZI], PLUS0,PLUS1, PLUS0,PLUS0, PLUS0,PLUS1);

        const REAL lapse_Psi2_iphkph = avg(INTERP_VARS[LAPSE_PSI2I], PLUS0,PLUS1, PLUS0,PLUS0, PLUS0,PLUS1);
        //REAL lapse_iphkph = avg(INTERP_VARS[LAPM1I], PLUS0,PLUS1, PLUS0,PLUS0, PLUS0,PLUS1)+1.0;
        //REAL psi_iphkph   = avg(INTERP_VARS[PSII  ], PLUS0,PLUS1, PLUS0,PLUS0, PLUS0,PLUS1);

        const REAL A_x_iphkph   = avg(INTERP_VARS[A_XI], PLUS0,PLUS1,MINUS1,PLUS0, PLUS0,PLUS0); // @ (i,j+1/2,k+1/2)
        const REAL A_y_iphkph   = avg(INTERP_VARS[A_YI], PLUS0,PLUS0, PLUS0,PLUS0, PLUS0,PLUS0); // @ (i+1/2,j,k+1/2)
        const REAL A_z_iphkph   = avg(INTERP_VARS[A_ZI], PLUS0,PLUS0,MINUS1,PLUS0, PLUS0,PLUS1); // @ (i+1/2,j+1/2,k)

        alpha_sqrtg_Ay_interp[index] = lapse_Psi2_iphkph*
          ( gupxy_iphkph*A_x_iphkph + gupyy_iphkph*A_y_iphkph + gupyz_iphkph*A_z_iphkph );


Appending to GiRaFFE_standalone_Ccodes/RHSs/Lorenz_psi6phi_rhs__add_gauge_terms_to_A_i_rhs.h


As above, except we are now calculating $\alpha \sqrt{\gamma} A^z = \alpha \sqrt{\gamma} (\gamma^{zx} A_x + \gamma^{zy} A_y + \gamma^{zz} A_z)$ at every gridpoint, making the appropriate cyclic permutations.

In [10]:
%%writefile -a $Ccodesdir/Lorenz_psi6phi_rhs__add_gauge_terms_to_A_i_rhs.h

        // DO A^Z TERM (interpolate to (i+1/2,j+1/2,k) )
        // \alpha \sqrt{\gamma} A^z = \alpha psi^6 A^z (RHS of \partial_i psi6phi)
        // Note that gupij is \tilde{\gamma}^{ij}, so we need to multiply by \psi^{-4}.
        const REAL gupxz_iphjph = avg(INTERP_VARS[GUPXZI], PLUS0,PLUS1, PLUS0,PLUS1, PLUS0,PLUS0);
        const REAL gupyz_iphjph = avg(INTERP_VARS[GUPYZI], PLUS0,PLUS1, PLUS0,PLUS1, PLUS0,PLUS0);
        const REAL gupzz_iphjph = avg(INTERP_VARS[GUPZZI], PLUS0,PLUS1, PLUS0,PLUS1, PLUS0,PLUS0);
        //REAL lapse_iphjph = avg(INTERP_VARS[LAPM1I], PLUS0,PLUS1, PLUS0,PLUS1, PLUS0,PLUS0)+1.0;
        //REAL psi_iphjph   = avg(INTERP_VARS[PSII  ], PLUS0,PLUS1, PLUS0,PLUS1, PLUS0,PLUS0);

        const REAL lapse_Psi2_iphjph = avg(INTERP_VARS[LAPSE_PSI2I], PLUS0,PLUS1, PLUS0,PLUS1, PLUS0,PLUS0);

        const REAL A_x_iphjph   = avg(INTERP_VARS[A_XI], PLUS0,PLUS1, PLUS0,PLUS0,MINUS1,PLUS0); // @ (i,j+1/2,k+1/2)
        const REAL A_y_iphjph   = avg(INTERP_VARS[A_YI], PLUS0,PLUS0, PLUS0,PLUS1,MINUS1,PLUS0); // @ (i+1/2,j,k+1/2)
        const REAL A_z_iphjph   = avg(INTERP_VARS[A_ZI], PLUS0,PLUS0, PLUS0,PLUS0, PLUS0,PLUS0); // @ (i+1/2,j+1/2,k)

        alpha_sqrtg_Az_interp[index] = lapse_Psi2_iphjph*
          ( gupxz_iphjph*A_x_iphjph + gupyz_iphjph*A_y_iphjph + gupzz_iphjph*A_z_iphjph );



Appending to GiRaFFE_standalone_Ccodes/RHSs/Lorenz_psi6phi_rhs__add_gauge_terms_to_A_i_rhs.h


<a id='a_gauge_term'></a>

### Step 2.b.iii: Compute $\alpha \Phi - \beta^j A_j$ \[Back to [top](#toc)\]
$$\label{a_gauge_term}$$

Now, we will calculate $\alpha \Phi - \beta^j A_j$ at every gridpoint for the $A_k$ evolution equation. This follows fairly similarly to what we did above, except we must interpolate all the inputs (lapse $\alpha$, shift $\beta^i$, vector potential $A_i$) to the cell vertices $(i+1/2,j+1/2,k+1/2)$. While $\psi^6\Phi$ is already sampled at cell vertices, the equation specifically calls for only $\Phi$; fortunately, we have precomputed $\alpha/\psi^6$ at cell centers.

Thus, we ultimately will be interpolating 
* the shift and $\alpha/\psi^6$ from the cell centers to the cell vertices, using indices `i,j,k`, `i+1,j,k`, `i,j+1,k`, `i,j,k+1`, `i,j+1,k+1`, `i+1,j,k+1`, `i+1,j+1,k`, and `i+1,j+1,k+1`
* $A_x$ from the cell edge $(i,j+1/2,k+1/2)$ using `i,j,k` and `i+1,j,k`
* $A_y$ from the cell edge $(i+1/2,j,k+1/2)$ using `i,j,k` and `i,j+1,k`
* $A_z$ from the cell edge $(i+1/2,j+1/2,k)$ using `i,j,k` and `i,j,k+1`

In [11]:
%%writefile -a $Ccodesdir/Lorenz_psi6phi_rhs__add_gauge_terms_to_A_i_rhs.h

        // Next set \alpha \Phi - \beta^j A_j at (i+1/2,j+1/2,k+1/2):
        //   We add a "L" suffix to shifti_iphjphkph to denote "LOCAL", as we set
        //      shifti_iphjphkph[] gridfunction below.
        const REAL shiftx_iphjphkphL = avg(INTERP_VARS[SHIFTXI], PLUS0,PLUS1, PLUS0,PLUS1, PLUS0,PLUS1);
        const REAL shifty_iphjphkphL = avg(INTERP_VARS[SHIFTYI], PLUS0,PLUS1, PLUS0,PLUS1, PLUS0,PLUS1);
        const REAL shiftz_iphjphkphL = avg(INTERP_VARS[SHIFTZI], PLUS0,PLUS1, PLUS0,PLUS1, PLUS0,PLUS1);
        const REAL lapse_over_Psi6_iphjphkphL = avg(INTERP_VARS[LAPSE_OVER_PSI6I], PLUS0,PLUS1, PLUS0,PLUS1, PLUS0,PLUS1);
        //REAL psi_iphjphkph = avg(INTERP_VARS[PSII  ], PLUS0,PLUS1, PLUS0,PLUS1, PLUS0,PLUS1);
        //REAL psi2_iphjphkph= psi_iphjphkph*psi_iphjphkph;
        //REAL psi6_iphjphkph= psi2_iphjphkph*psi2_iphjphkph*psi2_iphjphkph;
        const REAL A_x_iphjphkph = avg(INTERP_VARS[A_XI], PLUS0,PLUS1, PLUS0,PLUS0, PLUS0,PLUS0); // @ (i,j+1/2,k+1/2)
        const REAL A_y_iphjphkph = avg(INTERP_VARS[A_YI], PLUS0,PLUS0, PLUS0,PLUS1, PLUS0,PLUS0); // @ (i+1/2,j,k+1/2)
        const REAL A_z_iphjphkph = avg(INTERP_VARS[A_ZI], PLUS0,PLUS0, PLUS0,PLUS0, PLUS0,PLUS1); // @ (i+1/2,j+1/2,k)

        alpha_Phi_minus_betaj_A_j_iphjphkph[index] = psi6phi[index]*lapse_over_Psi6_iphjphkphL
          - (shiftx_iphjphkphL*A_x_iphjphkph + shifty_iphjphkphL*A_y_iphjphkph + shiftz_iphjphkphL*A_z_iphjphkph);


Appending to GiRaFFE_standalone_Ccodes/RHSs/Lorenz_psi6phi_rhs__add_gauge_terms_to_A_i_rhs.h


We will also store the shift function at $(i+1/2,j+1/2,k+1/2)$ (which we have already interpolated) for $\partial_j \beta^j [\psi^6 \Phi]$, which is part of the $\psi^6 \Phi$ evolution equation.

In [12]:
%%writefile -a $Ccodesdir/Lorenz_psi6phi_rhs__add_gauge_terms_to_A_i_rhs.h

        // Finally, save shifti_iphjphkph, for \partial_j \beta^j psi6phi
        shiftx_iphjphkph[index]=shiftx_iphjphkphL;
        shifty_iphjphkph[index]=shifty_iphjphkphL;
        shiftz_iphjphkph[index]=shiftz_iphjphkphL;
      }


Appending to GiRaFFE_standalone_Ccodes/RHSs/Lorenz_psi6phi_rhs__add_gauge_terms_to_A_i_rhs.h


<a id='finite_difference'></a>

## Step 2.c: Take the finite difference derivatives \[Back to [top](#toc)\]
$$\label{finite_difference}$$

We restate the equations we want to solve:
* $\partial_t A_i -= \partial_i (\alpha \Phi - \beta^j A_j)$
* $\partial_t [\sqrt{\gamma} \Phi] + \partial_j (\alpha\sqrt{\gamma}A^j - \beta^j [\sqrt{\gamma} \Phi]) = - \xi \alpha [\sqrt{\gamma} \Phi]$

So far, we have calculated only the operands of the differentiation operator, i.e. the parts within the parentheses as they are written above, and repeated here:
* $\alpha \Phi - \beta^j A_j$
* $\alpha\sqrt{\gamma}A^j - \beta^j [\sqrt{\gamma} \Phi]$

So, we must now take the derivatives using finite-difference methods. We will thus loop over the grid interior, and define the index to which we will be writing our results.

In [13]:
%%writefile -a $Ccodesdir/Lorenz_psi6phi_rhs__add_gauge_terms_to_A_i_rhs.h

  // This loop requires two additional ghostzones in every direction. Hence the following loop definition:
#pragma omp parallel for
  for(int k=NGHOSTS;k<Nxx_plus_2NGHOSTS2-NGHOSTS;k++) for(int j=NGHOSTS;j<Nxx_plus_2NGHOSTS1-NGHOSTS;j++) for(int i=NGHOSTS;i<Nxx_plus_2NGHOSTS0-NGHOSTS;i++) {
        const int index = IDX3S(i,j,k);


Appending to GiRaFFE_standalone_Ccodes/RHSs/Lorenz_psi6phi_rhs__add_gauge_terms_to_A_i_rhs.h


<a id='a_rhs'></a>

### Step 2.c.i: Compute the $A_i$ gauge term \[Back to [top](#toc)\]
$$\label{a_rhs}$$

We start with the right hand side of $\partial_t A_i$, adding on the gauge term $-\partial_i (\alpha \Phi - \beta^j A_j)$. Recall that as we calculated the operand, we interpolated it to the cell vertices $(i+1/2,j+1/2,k+1/2)$. Thus, since $A_x$ is sampled at $(i,j+1/2,k+1/2)$, the relevant finite-difference template here is 
$$-(Q_{i,j,k} - Q_{i-1,j,k})/dx = (Q_{i-1,j,k} - Q_{i,j,k})/dx$$
for a dummy quantity $Q$, and so on for the $y$ and $z$ directions.

In [14]:
%%writefile -a $Ccodesdir/Lorenz_psi6phi_rhs__add_gauge_terms_to_A_i_rhs.h

        // \partial_t A_i = [reconstructed stuff] + [gauge stuff],
        //    where [gauge stuff] = -\partial_i (\alpha \Phi - \beta^j A_j)
        const REAL alpha_Phi_minus_betaj_A_j_iphjphkphL = alpha_Phi_minus_betaj_A_j_iphjphkph[index];
        // - partial_i -> - (A_{i} - A_{i-1})/dX = (A_{i-1} - A_{i})/dX, for Ax
        Ax_rhs[index] += invdx0*(alpha_Phi_minus_betaj_A_j_iphjphkph[IDX3S(i-1,j,k)] - alpha_Phi_minus_betaj_A_j_iphjphkphL);
        Ay_rhs[index] += invdx1*(alpha_Phi_minus_betaj_A_j_iphjphkph[IDX3S(i,j-1,k)] - alpha_Phi_minus_betaj_A_j_iphjphkphL);
        Az_rhs[index] += invdx2*(alpha_Phi_minus_betaj_A_j_iphjphkph[IDX3S(i,j,k-1)] - alpha_Phi_minus_betaj_A_j_iphjphkphL);


Appending to GiRaFFE_standalone_Ccodes/RHSs/Lorenz_psi6phi_rhs__add_gauge_terms_to_A_i_rhs.h


<a id='phi_rhs'></a>

### Step 2.c.ii: Compute the $[\psi^6 \Phi]$ gauge term \[Back to [top](#toc)\]
$$\label{phi_rhs}$$

The operand of the gradient operator in the $\partial_t [\psi^6 \Phi]$ equation is the sum of two terms; we will first compute the shift advection term $\partial_j (\beta^j \psi^6 \Phi)$.

In [15]:
%%writefile -a $Ccodesdir/Lorenz_psi6phi_rhs__add_gauge_terms_to_A_i_rhs.h

        // \partial_t psi6phi = [shift advection term] + \partial_j (\alpha \sqrt{\gamma} A^j)
        // Here we compute [shift advection term] = \partial_j (\beta^j psi6phi)
        // Cache misses are likely more expensive than branch mispredictions here,
        //       which is why we use if() statements and array lookups inside the if()'s.
        REAL psi6phi_rhsL=0.0;
        const REAL psi6phiL=psi6phi[index];
        const REAL shiftx_iphjphkphL=shiftx_iphjphkph[index];
        const REAL shifty_iphjphkphL=shifty_iphjphkph[index];
        const REAL shiftz_iphjphkphL=shiftz_iphjphkph[index];

        // \partial_x (\beta^x psi6phi) :
        if(shiftx_iphjphkphL < 0.0) {
          psi6phi_rhsL+=0.5*invdx0*(+    shiftx_iphjphkph[IDX3S(i-2,j,k)]*psi6phi[IDX3S(i-2,j,k)]
                                  -4.0*shiftx_iphjphkph[IDX3S(i-1,j,k)]*psi6phi[IDX3S(i-1,j,k)]
                                  +3.0*shiftx_iphjphkphL*                               psi6phiL);
        } else {
          psi6phi_rhsL+=0.5*invdx0*(-    shiftx_iphjphkph[IDX3S(i+2,j,k)]*psi6phi[IDX3S(i+2,j,k)]
                                  +4.0*shiftx_iphjphkph[IDX3S(i+1,j,k)]*psi6phi[IDX3S(i+1,j,k)]
                                  -3.0*shiftx_iphjphkphL*                               psi6phiL);
        }

        // \partial_y (\beta^y psi6phi) :
        if(shifty_iphjphkphL < 0.0) {
          psi6phi_rhsL+=0.5*invdx1*(+    shifty_iphjphkph[IDX3S(i,j-2,k)]*psi6phi[IDX3S(i,j-2,k)]
                                  -4.0*shifty_iphjphkph[IDX3S(i,j-1,k)]*psi6phi[IDX3S(i,j-1,k)]
                                  +3.0*shifty_iphjphkphL*                               psi6phiL);
        } else {
          psi6phi_rhsL+=0.5*invdx1*(-    shifty_iphjphkph[IDX3S(i,j+2,k)]*psi6phi[IDX3S(i,j+2,k)]
                                  +4.0*shifty_iphjphkph[IDX3S(i,j+1,k)]*psi6phi[IDX3S(i,j+1,k)]
                                  -3.0*shifty_iphjphkphL*                               psi6phiL);
        }

        // \partial_z (\beta^z psi6phi) :
        if(shiftz_iphjphkphL < 0.0) {
          psi6phi_rhsL+=0.5*invdx2*(+    shiftz_iphjphkph[IDX3S(i,j,k-2)]*psi6phi[IDX3S(i,j,k-2)]
                                  -4.0*shiftz_iphjphkph[IDX3S(i,j,k-1)]*psi6phi[IDX3S(i,j,k-1)]
                                  +3.0*shiftz_iphjphkphL*                               psi6phiL);
        } else {
          psi6phi_rhsL+=0.5*invdx2*(-    shiftz_iphjphkph[IDX3S(i,j,k+2)]*psi6phi[IDX3S(i,j,k+2)]
                                  +4.0*shiftz_iphjphkph[IDX3S(i,j,k+1)]*psi6phi[IDX3S(i,j,k+1)]
                                  -3.0*shiftz_iphjphkphL*                               psi6phiL);
        }


Appending to GiRaFFE_standalone_Ccodes/RHSs/Lorenz_psi6phi_rhs__add_gauge_terms_to_A_i_rhs.h


The operand of the gradient operator in the $\partial_t [\psi^6 \Phi]$ equation is the sum of two terms; we will add the term $-\partial_j (\alpha \sqrt{\gamma} A^j)$. Recall that we computed $\alpha \sqrt{\gamma} A^x$ at $(i,j+1/2,k+1/2)$; thus, the appropriate finite-difference template here is 
$$-(Q_{i+1,j,k} - Q_{i,j,k})/dx = (Q_{i,j,k} - Q_{i+1,j,k})/dx$$
for a dummy quantity $Q$, and so on for the $y$ and $z$ directions.

Finally, we will add the damping factor to $\partial_t [\psi^6 \Phi]$, which is given as $\xi \alpha \psi^6 \Phi$, where $\xi$ is some scalar damping parameter. Since we interpolated the lapse $\alpha$ to cell vertices earlier, everything is exactly where we need it!

In [16]:
%%writefile -a $Ccodesdir/Lorenz_psi6phi_rhs__add_gauge_terms_to_A_i_rhs.h

        // Next we add \partial_j (\alpha \sqrt{\gamma} A^j) to \partial_t psi6phi:
        psi6phi_rhsL+=invdx0*(alpha_sqrtg_Ax_interp[index] - alpha_sqrtg_Ax_interp[IDX3S(i+1,j,k)])
          +           invdx1*(alpha_sqrtg_Ay_interp[index] - alpha_sqrtg_Ay_interp[IDX3S(i,j+1,k)])
          +           invdx2*(alpha_sqrtg_Az_interp[index] - alpha_sqrtg_Az_interp[IDX3S(i,j,k+1)]);

        // *GENERALIZED* LORENZ GAUGE:
        // Finally, add damping factor to \partial_t psi6phi
        //subtract lambda * alpha psi^6 Phi
        psi6phi_rhsL+=-xi_damping*alpha_iphjphkph[index]*psi6phiL;

        psi6phi_rhs[index] = psi6phi_rhsL;
      }
}


Appending to GiRaFFE_standalone_Ccodes/RHSs/Lorenz_psi6phi_rhs__add_gauge_terms_to_A_i_rhs.h


<a id='interp_func'></a>

## Step 2.d: The interpolation function \[Back to [top](#toc)\]
$$\label{interp_func}$$

Here, we give the function definition for our interpolator, `avg`. It takes the arithmetic mean of the points of the gridfunction `f` specified by `imin`, `imax`, `jmin`, `jmax`, `kmin`, and `kmax`.

In [17]:
%%writefile -a $Ccodesdir/Lorenz_psi6phi_rhs__add_gauge_terms_to_A_i_rhs.h

static inline REAL avg(const REAL f[PLUS2+1][PLUS2+1][PLUS2+1],const int imin,const int imax, const int jmin,const int jmax, const int kmin,const int kmax) {
  REAL retval=0.0,num_in_sum=0.0;
  for(int kk=kmin;kk<=kmax;kk++) for(int jj=jmin;jj<=jmax;jj++) for(int ii=imin;ii<=imax;ii++) {
        retval+=f[kk][jj][ii]; num_in_sum++;
      }
  return retval/num_in_sum;
}


Appending to GiRaFFE_standalone_Ccodes/RHSs/Lorenz_psi6phi_rhs__add_gauge_terms_to_A_i_rhs.h


<a id='code_validation'></a>

# Step 3: Code Validation \[Back to [top](#toc)\]
$$\label{code_validation}$$

To validate the code in this tutorial we check for agreement between the files

1. that were written in this tutorial and
1. those that are generated by the python module


In [18]:
# Define the directory that we wish to validate against:
valdir = "GiRaFFE_NRPy/GiRaFFE_Ccode_library/RHSs/"

import GiRaFFE_NRPy.GiRaFFE_NRPy_staggered_Source_Terms as source
source.GiRaFFE_NRPy_Source_Terms(valdir)

import difflib
import sys

print("Printing difference between original C code and this code...")
# Open the files to compare
files = ["Lorenz_psi6phi_rhs__add_gauge_terms_to_A_i_rhs.h"]

for file in files:
    print("Checking file " + file)
    with open(os.path.join(valdir,file)) as file1, open(os.path.join(Ccodesdir,file)) as file2:
        # Read the lines of each file
        file1_lines = file1.readlines()
        file2_lines = file2.readlines()
        num_diffs = 0
        for line in difflib.unified_diff(file1_lines, file2_lines, fromfile=os.path.join(valdir+file), tofile=os.path.join(Ccodesdir+file)):
            sys.stdout.writelines(line)
            num_diffs = num_diffs + 1
        if num_diffs == 0:
            print("No difference. TEST PASSED!")
        else:
            print("ERROR: Disagreement found with .py file. See differences above.")
            sys.exit(1)

Printing difference between original C code and this code...
Checking file Lorenz_psi6phi_rhs__add_gauge_terms_to_A_i_rhs.h
No difference. TEST PASSED!


<a id='latex_pdf_output'></a>

# Step 4: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-GiRaFFE_NRPy_staggered-Source_Terms.pdf](Tutorial-GiRaFFE_NRPy_staggered-Source_Terms.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [19]:
import cmdline_helper as cmd    # NRPy+: Multi-platform Python command-line interface
cmd.output_Jupyter_notebook_to_LaTeXed_PDF("Tutorial-GiRaFFE_NRPy_staggered-Source_Terms",location_of_template_file=os.path.join(".."))

Created Tutorial-GiRaFFE_NRPy_staggered-Source_Terms.tex, and compiled
    LaTeX file to PDF file Tutorial-GiRaFFE_NRPy_staggered-Source_Terms.pdf
